<a href="https://colab.research.google.com/github/MelqonyanG/ML_Intro/blob/master/ACA_ML_8_4_Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACA ML 8.4 - Machine translation

This notebook is part of the [Machine Learning Course](https://aca.am/en/machine-learning/) at Armenian Code Academy, and accompanies lectures slides and lectures nots for the 2-week block on NLP. This notebook supplements the lecture on machine translation, focusing on a couple of practical aspects such as how to perform beam search (important for the decoding stage of the translation model).

Created: March 28, 2021 | Last major update: March 28, 2021

Author: [Vahe Tshitoyan](https://vtshitoyan.github.io) (please reach out with any questions or suggestions)

**A comprehensive tutorial can be found [here](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb?authuser=1) for a detailed example of using Attention for machine translation.**

In [ ]:
import numpy as np
import tensorflow as tf

## Beam search

In [ ]:
# Load the model and the vocabulary (see Lecture 8.3 on how to train the model).
model = tf.keras.models.load_model('arm_rnn_language_model.h5')
with open('idx2word.txt', 'r', encoding='utf-8') as idx2word_file:
  idx2word = [w.strip() for w in idx2word_file.readlines()]
word2idx = {w: i for i, w in enumerate(idx2word)}

In [ ]:
def generate_text(input_sentence, nr_extra_words, method='sample'):
  """Generates text either by sampling or using a greedy approach (argmax)."""
  # Converts words to their indexes in the vocabulary.
  encoded_input = [word2idx[w] if w in word2idx else 1 for w in input_sentence]
  for _ in range(nr_extra_words):
    next_word_distribution = model.predict([encoded_input])[0]
    if method == 'sample':
      next_word = np.random.choice(
          range(len(idx2word)), 
          p=next_word_distribution)
    elif method == 'greedy':
      next_word = np.argsort(next_word_distribution)[-1]
    else:
      raise ValueError(
          f'Unsupported method {method}. Use "greedy" or "sample".')
    # Add the new word to the sequence to continue generating text.
    encoded_input.append(next_word)
  return [idx2word[encoded_word] for encoded_word in encoded_input]

In [ ]:
def generate_text_beam_search(input_sentence, nr_extra_words, beam_k=10):
  """Generates text using beam search."""
  initial_length = len(input_sentence)
  encoded_input = [word2idx[w] if w in word2idx else 1 for w in input_sentence]
  candidates = [(encoded_input, 0)]
  for _ in range(nr_extra_words):
    new_candidates = []
    for candidate, score in candidates:
      next_word_distribution = model.predict([candidate])[0]
      top_k_words = np.argsort(next_word_distribution)[-beam_k:]
      top_k_scores = np.log(next_word_distribution[top_k_words])
      for w, s in zip(top_k_words, top_k_scores):
        new_candidates.append((candidate + [w], score + s))
    candidates = sorted(new_candidates, key=lambda x: x[1])[-beam_k:]
  return [idx2word[encoded_word] for encoded_word in candidates[-1][0]]

In [ ]:
' '.join(generate_text(['ՀՀ'], 50, 'sample'))

'ՀՀ Լին դեմ էին հետո աշխատել , որոնք է , երբ սակրավորները ստեղծված , միայն փոխհատուցվում եւ պայմանների համար ։ Նախնական 1,5 հետոյի գործով ինքնությունը չպարզած ֆաունայի գտածոյի որոշումից մեկի ընկեր , ԱԺ-ին զգուշացրել է ։ Այն բերել ՔԿՀ-ի կալանավորվածներին պահելու վայրի խցերում արձանագրվել են ՀՀ ԱՆ քրեակատարողական վերահսկողության ծառայության պետ'

In [ ]:
' '.join(generate_text(['ՀՀ'], 50, 'greedy'))

'ՀՀ գլխավոր դատախազության վերոնշյալ վարչության պետը անցել են , որ նույն օրը , ժամը , երբ կաթոլիկները , իսկ պահվող անձանց տրամադրվող զբոսանքի ժամանակցույց չունենալուն ։ Արդյոք գյուղի ՝ Մանիլայից 60 տոկոսը ։ - չլինելը որևէ տարբերակի պատասխանատվություն չի եղել , որ իր էջում գրել է , որ ապրիլի 22-ին ,'

In [ ]:
' '.join(generate_text_beam_search(['Փաշինյանի'], 50))

'Փաշինյանի նկատմամբ հսկողության վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական վարչության Շենգավիթ վարչական շրջանի քննչական բաժնում քննվող քրեական գործից անջատված մասով նախաքննությունն ավարտվել'